In [11]:
import numpy as np
import random

def sigmoide(z):
    """La fonction d'activation sigmoide"""
    return 1.0/(1.0+np.exp(-z))

def sigmoide_derivee(z):
    """Derivative of the sigmoid function."""
    return sigmoide(z)*(1-sigmoide(z))

class RNA(object):

    def __init__(self, nc):
        """ nc[c] contient le nombre de neurones de la couche c, c = 0 ...nombre_couches-1
        la couche d'indice 0 est la couche d'entrée
        w[c] est la marice des poids entre la couche c et c+1
        w[c][i,j] est le poids entre le neuronne i de la couche c et j de la couche c+1
        i = 0 correspond au biais par convention
        les poids sont initialisés avec un nombre aléatoire selon une distribution N(0,1)
        """
        self.nombre_couches = len(nc)
        self.nc = nc
        np.random.seed(42)
        self.w = [np.random.randn(x+1, y) for x, y in zip(nc[:-1], nc[1:])]
        print("nc:",nc)
        print("zip(nc[:-1], nc[1:])",zip(nc[:-1], nc[1:]))
        print("w:",self.w)

    def propagation_avant(self, a):
        """a est un vecteur d'activation. a[0]=1 correspond au biais"""
        for wc in self.w:
            a = np.vstack((np.ones(1),sigmoide(np.dot(wc.transpose(), a))))
        return a

    def SGD(self, donnees_entrainement_x, donnees_entrainement_y, epochs, taille_mini_batch, eta,
            donnees_test_x, donnees_test_y):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``donnees_entrainement`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``donnees_test`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        
        n = len(donnees_entrainement_x)
        donnees_entrainement=[(donnees_entrainement_x[i],donnees_entrainement_y[i]) for i in range(n)]
        n_test = len(donnees_test_x)
        donnees_test=[(donnees_test_x[i],donnees_test_y[i]) for i in range(n_test)]

        for j in range(epochs):
            random.shuffle(donnees_entrainement)
            mini_batches = [
                donnees_entrainement[k:k+taille_mini_batch]
                for k in range(0, n, taille_mini_batch)]
            for mini_batch in mini_batches:
                self.mini_batch(mini_batch, eta)
            print ("Epoch {0}: {1} / {2}".format(j, self.evaluate(donnees_test), n_test))

    def mini_batch(self, mini_batch, eta):
        """Update the network's w and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_w = [np.zeros(w.shape) for w in self.w]
        for x, y in mini_batch:
            delta_nabla_w = self.retropropagation(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.w = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.w, nabla_w)]

    def retropropagation(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.w``."""
        # print(x,y)
        # print(type(x))
        # print(type(y))
        nabla_w = [np.zeros(wc.shape) for wc in self.w]
        # propagation avant avec stockage des activations[0] est à 1 pour les biais
        activation = np.vstack((np.ones(1),x.reshape(x.size,1))) # activation
        activations = [np.vstack((np.ones(1),x.reshape(x.size,1)))] # liste des activations couche par couche
        zs = [] # # liste des z couche par couche

        for wc in self.w:
            z = np.dot(wc.transpose(), activation)
            zs.append(z)
            activation = np.vstack((np.ones(1),sigmoide(z))) 
            activations.append(activation)
        # print("zs:",zs)
        # print("activations:",activations)
        
        # rétropropagation 
        # delta est le vecteur des dérivées par rapport zjc-1
        # calcul de la dérivée par rapport a z pour la couche de sortie
        delta = derivee_cout(activations[-1][1:], y) * sigmoide_derivee(zs[-1])
        nabla_w[-1] = np.dot(activations[-2],delta.transpose())
        # print("del w couche[2]:",nabla_w[-1])

        for l in range(2, self.nombre_couches):
            z = zs[-l]
            sp = sigmoide_derivee(z)
            delta = np.dot(self.w[-l+1], delta)[1:] * sp
            # print("z:",z)
            # print("sp:",sp)
            # print("delta:",delta)
            # print("activations:",activations)
            # print("l:",l)
            # print("activations[-l-1]:",activations[-l-1])
            nabla_w[-l] = np.dot(activations[-l-1],delta.transpose())
        return nabla_w

    def evaluate(self, donnees_test):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.propagation_avant(np.vstack((np.ones(1),x.reshape(x.size,1))))), y)
                        for (x, y) in donnees_test]
        somme_carres=0
        for (x, y) in donnees_test:
            y_prediction = self.propagation_avant(np.vstack((np.ones(1),x.reshape(x.size,1))))
#            print("y_prediction:",y_prediction,"y:",y)
            somme_carres=somme_carres+(y_prediction[1]-y)**2
        return somme_carres    
#         return sum((x-y)**2 for (x, y) in test_results)

def derivee_cout(output_activations, y):
    """ Vecteur des dérivées de la fonction de cout vs aj"""
    return (output_activations-y)

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

maisons_boston = datasets.load_boston()
maisons_X = maisons_boston.data
maisons_y = maisons_boston.target

def diviser_ent_test(donnees, proportion):
    donnees_permutes = np.random.permutation(donnees)
    taille_test = int(len(donnees) * proportion)
    return donnees[:taille_test],donnees[taille_test:]

# Diviser les données
X_test,X_ent = diviser_ent_test(maisons_X, 0.8)
Y_test,Y_ent = diviser_ent_test(maisons_y, 0.8)


un_rna = RNA([13, 5, 1])
un_rna.SGD(X_ent,Y_ent, 5, 50, 1.0,X_test,Y_test)


nc: [13, 5, 1]
zip(nc[:-1], nc[1:]) <zip object at 0x0000023241FAD188>
w: [array([[ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986, -0.23415337],
       [-0.23413696,  1.57921282,  0.76743473, -0.46947439,  0.54256004],
       [-0.46341769, -0.46572975,  0.24196227, -1.91328024, -1.72491783],
       [-0.56228753, -1.01283112,  0.31424733, -0.90802408, -1.4123037 ],
       [ 1.46564877, -0.2257763 ,  0.0675282 , -1.42474819, -0.54438272],
       [ 0.11092259, -1.15099358,  0.37569802, -0.60063869, -0.29169375],
       [-0.60170661,  1.85227818, -0.01349722, -1.05771093,  0.82254491],
       [-1.22084365,  0.2088636 , -1.95967012, -1.32818605,  0.19686124],
       [ 0.73846658,  0.17136828, -0.11564828, -0.3011037 , -1.47852199],
       [-0.71984421, -0.46063877,  1.05712223,  0.34361829, -1.76304016],
       [ 0.32408397, -0.38508228, -0.676922  ,  0.61167629,  1.03099952],
       [ 0.93128012, -0.83921752, -0.30921238,  0.33126343,  0.97554513],
       [-0.47917424, -0.18565898, -1.

In [9]:
maisons_X.head()




AttributeError: 'numpy.ndarray' object has no attribute 'head'